In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.external import *
from local.data.pipeline import *
from local.text.core import *
from local.notebook.showdoc import show_doc

NameError: name 'Transform' is not defined

In [ ]:
#default_exp text.data
#default_cls_lvl 3

In [ ]:
#hide
torch.cuda.set_device(int(os.environ.get('DEFAULT_GPU') or 0))

# Text data

> Functions and transforms to help gather text data in a `DataSource

## Numericalizing

In [ ]:
#export
class Text():
    @staticmethod
    def show(o, ctx=None, sep=None, **kwargs): 
        sep = defaults.text_token_sep
        return show_title(sep.join(o), ctx=ctx)

In [ ]:
# export
class Numericalize(MultiCategorize):
    "Reversible transform of multi-category strings to `vocab` id"
    def __init__(self, vocab=None, subset_idx=None, min_freq=3, max_vocab=60000):
        super().__init__(vocab=vocab, subset_idx=subset_idx)
        self.min_freq,self.max_vocab = min_freq,max_vocab
    def setup(self, dsrc):
        if not dsrc: return
        if self.vocab is None:
            dsrc = dsrc.train if self.subset_idx is None else dsrc.subset(self.subset_idx)
            vals = set()
            for b in dsrc: vals = vals.union(set(b))
            self.vocab,self.otoi = uniqueify(list(vals), sort=True, bidir=True)
        setattr_parent(dsrc, 'vocab', self.vocab)

    def encodes(self, o)->Text: return [self.otoi[o_] for o_ in o]
    def decodes(self, o):                return [self.vocab[o_] for o_ in o]